# LangChain Tool, Tool Calling, Agent 튜토리얼

이 노트북은 LangChain의 핵심 개념인 **Tool**, **Tool Calling**, **Agent**를 단계별로 학습할 수 있도록 구성되었습니다.

## 📚 학습 목차

1. **Tool 기초** - 첫 번째 Tool 만들기
2. **Tool Calling** - LLM이 Tool을 호출하는 방법
3. **Agent** - 복잡한 문제를 단계별로 해결하기

---

## 🎯 학습 목표

이 노트북을 완료하면:
- Tool을 직접 만들 수 있습니다
- LLM이 Tool을 자동으로 사용하도록 설정할 수 있습니다  
- 복잡한 업무를 처리하는 Agent를 개발할 수 있습니다

---

## ⚙️ 환경 설정

먼저 필요한 패키지를 설치하고 API 키를 설정해야 합니다.


In [2]:
# 필요한 패키지 설치 (주석 해제해서 실행)
# !pip install langchain langchain-openai python-dotenv

# 패키지 import
from langchain.tools import tool
from langchain_openai import ChatOpenAI
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage, SystemMessage
from typing import Annotated
import random
from datetime import datetime

# API 키 로드
from dotenv import load_dotenv
load_dotenv()

print("✅ 패키지 로드 완료!")
print("⚠️  OpenAI API 키가 설정되어 있는지 확인하세요!")
print("   (.env 파일에 OPENAI_API_KEY=your_key_here)")


✅ 패키지 로드 완료!
⚠️  OpenAI API 키가 설정되어 있는지 확인하세요!
   (.env 파일에 OPENAI_API_KEY=your_key_here)


# 2️⃣ Tool Calling

## Tool Calling이란?
**Tool Calling**은 LLM이 주어진 문제를 해결하기 위해 적절한 Tool을 선택하고 실행하는 과정입니다.

### 과정:
1. 🗣️ 사용자가 질문을 합니다
2. 🧠 LLM이 질문을 분석해서 어떤 Tool이 필요한지 판단
3. 🔧 LLM이 Tool을 호출하고 결과를 받음
4. 💬 LLM이 결과를 바탕으로 최종 답변 생성

이렇게 하면 LLM이 계산, 검색, 데이터 처리 등을 할 수 있게 됩니다!


In [3]:
# 🔄 노트북 실행 시 주의사항: 이전 Tool들과 이름이 겹치지 않도록 주의!

# Tool Calling용 Tool들 준비하기 (새로운 이름으로)
@tool
def math_calculator(
    operation: Annotated[str, "연산 종류: add, subtract, multiply, divide"], 
    a: Annotated[float, "첫 번째 숫자"], 
    b: Annotated[float, "두 번째 숫자"]
) -> float:
    """수학 계산을 수행하는 계산기입니다."""
    
    if operation == "add":
        result = a + b
        print(f"계산: {a} + {b} = {result}")
    elif operation == "subtract":
        result = a - b
        print(f"계산: {a} - {b} = {result}")
    elif operation == "multiply":
        result = a * b
        print(f"계산: {a} × {b} = {result}")
    elif operation == "divide":
        if b == 0:
            return "0으로 나눌 수 없습니다!"
        result = a / b
        print(f"계산: {a} ÷ {b} = {result}")
    else:
        return f"지원하지 않는 연산입니다: {operation}"
    
    return result

@tool
def user_database(name: Annotated[str, "사용자 이름"]) -> dict:
    """사용자 정보를 조회하는 가짜 데이터베이스입니다."""
    
    # 가짜 사용자 데이터베이스
    users_db = {
        "김철수": {"나이": 25, "직업": "개발자", "취미": "독서"},
        "이영희": {"나이": 30, "직업": "디자이너", "취미": "그림그리기"},
        "박민수": {"나이": 28, "직업": "데이터분석가", "취미": "영화감상"},
    }
    
    if name in users_db:
        result = users_db[name]
        print(f"{name}님의 정보를 찾았습니다: {result}")
        return result
    else:
        return {"오류": f"{name}님의 정보를 찾을 수 없습니다."}

@tool  
def time_checker() -> str:
    """현재 시간을 반환합니다."""
    now = datetime.now()
    time_str = now.strftime("%Y년 %m월 %d일 %H시 %M분")
    print(f"현재 시간: {time_str}")
    return time_str

print("✅ Tool Calling용 Tool들이 준비되었습니다!")
print("- math_calculator: 사칙연산")
print("- user_database: 사용자 정보 조회") 
print("- time_checker: 현재 시간")
print()
print("💡 중요: 노트북에서는 Tool 이름이 중복되지 않도록 주의하세요!")


✅ Tool Calling용 Tool들이 준비되었습니다!
- math_calculator: 사칙연산
- user_database: 사용자 정보 조회
- time_checker: 현재 시간

💡 중요: 노트북에서는 Tool 이름이 중복되지 않도록 주의하세요!


## LLM에 Tool 연결하기

이제 LLM이 이 Tool들을 사용할 수 있도록 연결해봅시다!


In [4]:
# LLM 모델 설정
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,  # 일관된 결과를 위해 0으로 설정
)

# Tool 목록 (새로운 이름으로 업데이트)
tools_for_calling = [math_calculator, user_database, time_checker]

# LLM에 Tool을 사용할 수 있도록 연결 ⭐ 이게 핵심!
llm_with_tools = llm.bind_tools(tools_for_calling)

print("🔗 LLM에 Tool들이 연결되었습니다!")
print("이제 LLM이 필요에 따라 Tool을 자동으로 사용할 수 있습니다!")


🔗 LLM에 Tool들이 연결되었습니다!
이제 LLM이 필요에 따라 Tool을 자동으로 사용할 수 있습니다!


## Tool Calling 실습

이제 LLM이 실제로 Tool을 호출하는지 확인해봅시다!


In [5]:
# 예시 1: 계산 요청
print("🔢 예시 1: 계산 요청")
user_question = "15에 23을 곱한 다음, 결과에서 47을 빼주세요"
print(f"사용자 질문: {user_question}")

# LLM에게 질문 전달
messages = [
    SystemMessage(content="당신은 도움이 되는 AI 어시스턴트입니다. 주어진 도구들을 사용해서 사용자의 질문에 답해주세요."),
    HumanMessage(content=user_question)
]

# LLM이 응답 생성 (Tool 호출 포함)
response = llm_with_tools.invoke(messages)

print(f"\nLLM 응답 내용: {response.content}")

# Tool 호출이 있는지 확인
if response.tool_calls:
    print(f"\n🔧 LLM이 호출한 Tool 개수: {len(response.tool_calls)}")
    
    for i, tool_call in enumerate(response.tool_calls, 1):
        print(f"\nTool 호출 {i}:")
        print(f"  - Tool 이름: {tool_call['name']}")
        print(f"  - 파라미터: {tool_call['args']}")
        
        # 실제로 Tool 실행
        for tool in tools_for_calling:
            if tool.name == tool_call['name']:
                result = tool.invoke(tool_call['args'])
                print(f"  - 실행 결과: {result}")
                break
else:
    print("Tool을 호출하지 않았습니다.")


🔢 예시 1: 계산 요청
사용자 질문: 15에 23을 곱한 다음, 결과에서 47을 빼주세요

LLM 응답 내용: 

🔧 LLM이 호출한 Tool 개수: 1

Tool 호출 1:
  - Tool 이름: math_calculator
  - 파라미터: {'operation': 'multiply', 'a': 15, 'b': 23}
계산: 15.0 × 23.0 = 345.0
  - 실행 결과: 345.0


In [ ]:
# 예시 2: 사용자 정보 조회
print("👤 예시 2: 사용자 정보 조회")
user_question = "김철수의 정보를 알려주세요"
print(f"사용자 질문: {user_question}")

messages = [
    SystemMessage(content="당신은 도움이 되는 AI 어시스턴트입니다. 주어진 도구들을 사용해서 사용자의 질문에 답해주세요."),
    HumanMessage(content=user_question)
]

response = llm_with_tools.invoke(messages)
print(f"응답 내용: {response.content}")

if response.tool_calls:
    print(f"\n🔧 LLM이 호출한 Tool:")
    for tool_call in response.tool_calls:
        print(f"  - Tool: {tool_call['name']}")
        print(f"  - 파라미터: {tool_call['args']}")
        
        # Tool 실행
        for tool in tools_for_calling:
            if tool.name == tool_call['name']:
                result = tool.invoke(tool_call['args'])
                print(f"  - 결과: {result}")
                break


## 🎯 Tool Calling 작동 원리

1. **🧠 LLM이 사용자 질문을 분석**
   - "15에 23을 곱한 다음, 47을 빼주세요"
   - → 이건 수학 계산이니까 calculator tool이 필요하겠네!

2. **🔧 LLM이 Tool 호출 계획 수립**
   - 첫 번째: calculator(operation="multiply", a=15, b=23)
   - 두 번째: calculator(operation="subtract", a=결과, b=47)

3. **⚡ Tool 실행**
   - 15 × 23 = 345
   - 345 - 47 = 298

4. **💬 최종 답변 생성**
   - "계산 결과는 298입니다."
